In [1]:
%load_ext autoreload
%autoreload 2

## DB Setup

We assume you already have a postgres database ready.

In [1]:
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "mysecretpassword"
HOST = "localhost"
PORT = 5432
TABLES = []  # list of tables to load or [] to load all tables

In [2]:
from db_connectors import PostgresConnector
from prompt_formatters import RajkumarFormatter, SqlCoderFormatter

# Get the connector and formatter
postgres_connector = PostgresConnector(
    user=USER, password=PASSWORD, dbname=DATABASE, host=HOST, port=PORT
)
postgres_connector.connect()
if len(TABLES) <= 0:
    TABLES.extend(postgres_connector.get_tables())

print(f"Loading tables: {TABLES}")

db_schema = [postgres_connector.get_schema(table) for table in TABLES]
formatter = SqlCoderFormatter(db_schema)

Loading tables: ['companies', 'stores', 'categories', 'products', 'inventory', 'restocks', 'transactions', 'productdemandforecast']


## Using LLama-2-7b-chat to enhance the SQL definition

In [2]:
pip install -q transformers pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!huggingface-cli login

In [1]:
%%time
from llm_utils import *
from prompt_formatters import SqlCoderFormatter
formatter = SqlCoderFormatter([])
# Reading from the file
with open("sample_table_str_raw.txt", 'r') as file:
    formatter.table_str = file.read()
from prompt_formatters import SqlCoderFormatter
formatter.table_str = add_comment_to_sql(formatter.table_str, debug=True)


<s>[INST] <<SYS>>
You are an assistant to a SQL developer. He gives a SQL database definition and you give him back the same SQL with comments that explain each column. For example : "company_id SERIAL PRIMARY KEY," becomes "company_id SERIAL PRIMARY KEY, -- company_id is the company unique ID". 
- It is important to keep all existing comments in the given SQL.
- Do not remove something from the SQL, just add to it.
- Add comments to all columns of each table.
- If a column name is an abreviation try to give a description of it.
<</SYS>>

Below is an SQL table definition: 
```sql
CREATE TABLE suppliers (
supplierid SERIAL PRIMARY KEY,
suppliername VARCHAR(100),
contactname VARCHAR(100),
address VARCHAR(255),
city VARCHAR(50),
country VARCHAR(50),
phone VARCHAR(15),
email VARCHAR(100)
)

CREATE TABLE products (
categoryid integer,
unitprice numeric,
stockquantity integer DEFAULT 0,
supplierid integer,
productid SERIAL PRIMARY KEY,
productname VARCHAR(100),
description text
)

CREATE TA

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tokenizing....
Generating...


In [4]:
print(formatter.table_str)


CREATE TABLE suppliers (
    supplierid SERIAL PRIMARY KEY, -- supplierid is the unique identifier for a supplier
    suppliername VARCHAR(100), -- supplier name
    contactname VARCHAR(100), -- contact person's name
    address VARCHAR(255), -- supplier's address
    city VARCHAR(50), -- city where the supplier is located
    country VARCHAR(50), -- country where the supplier is located
    phone VARCHAR(15), -- supplier's phone number
    email VARCHAR(100) -- supplier's email address
)

CREATE TABLE products (
    categoryid integer, -- categoryid is the unique identifier for a product category
    unitprice numeric, -- unit price of the product
    stockquantity integer DEFAULT 0, -- initial stock quantity of the product
    supplierid integer, -- supplier ID associated with the product
    productid SERIAL PRIMARY KEY, -- unique identifier for each product
    productname VARCHAR(100), -- product name
    description text -- product description
)

CREATE TABLE categories (
    ca

In [4]:
from llm_utils import *

formatter.table_str = add_comment_to_sql(formatter.table_str)
print(formatter.table_str)

/Users/mehditantaoui/Documents/Challenges/GenAI-SQL/NSQL/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...
Using device mps.


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.74s/it]


Tokenizing....
Generating...

CREATE TABLE companies (
company_id SERIAL PRIMARY KEY,
company_name VARCHAR(255) -- company_id is the company unique ID
)

CREATE TABLE stores (
company_id integer, -- foreign key referencing companies.company_id
store_id VARCHAR(255) PRIMARY KEY, -- store_id is the store unique ID
store_name VARCHAR(255), -- name of the store
address text, -- address of the store
phone_number VARCHAR(20) -- phone number of the store
)

CREATE TABLE categories (
category_id VARCHAR(255) PRIMARY KEY, -- category_id is the category unique ID
category_name_en VARCHAR(255) -- name of the category in English
)

CREATE TABLE products (
product_id VARCHAR(255) PRIMARY KEY, -- product_id is the product unique ID
article_id VARCHAR(255), -- article number of the product
product_name_en VARCHAR(255), -- name of the product in English
uom VARCHAR(50), -- unit of measurement for the product
buom VARCHAR(50), -- bought quantity of the product
category_id VARCHAR(255), -- foreign key r

In [5]:
# Writing to the file
with open("sample_table_str.txt", 'w') as file:
    file.write(formatter.table_str)

In [12]:
!pip install gradio typing-extensions

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 55.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 64.7 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/1

## Model Setup

In a separate screen or window, first install [Manifest](https://github.com/HazyResearch/manifest)
```bash
pip install manifest-ml\[all\]
```

Then run
```bash
python3 -m manifest.api.app \
    --model_type huggingface \
    --model_generation_type text-generation \
    --model_name_or_path NumbersStation/nsql-350M \
    --device 0
```

If successful, you will see an output like
```bash
* Running on http://127.0.0.1:5000
```

## Using Specialised LLM to answer question with SQL

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from llm_utils import find_device
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b")
model = AutoModelForCausalLM.from_pretrained("defog/sqlcoder-7b", torch_dtype=torch.float16)
device = find_device()
# Move the model to the device
model.to(device)

Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [11]:
%%time
import torch
torch.cuda.empty_cache()

from prompt_formatters import SqlCoderFormatter
formatter = SqlCoderFormatter([])
# Reading from the file
with open("sample_table_str.txt", 'r') as file:
    formatter.table_str = file.read()
print("Tokenizing....")
question = "What is the top seller in my products ?"
prompt = formatter.format_prompt(question, database='Postgres')
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Move the input to the MPS device
input_ids = input_ids.to(device)

print("Generating...")
generated_ids = model.generate(input_ids, max_length=2500)
response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Tokenizing....
Generating...


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.69 GiB total capacity; 22.60 GiB already allocated; 12.81 MiB free; 23.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [12]:
%%time
from llm_utils import prompt_llm_offline, HG_SQL_LLM_MODEL_PATH
from prompt_formatters import SqlCoderFormatter
formatter = SqlCoderFormatter([])
with open("sample_table_str.txt", 'r') as file:
    formatter.table_str = file.read()
question = "What is the most frequently used payment method ?"
prompt = formatter.format_prompt(question, database='Postgres')
response = prompt_llm_offline(prompt=prompt, max_length=2500, model_name=HG_SQL_LLM_MODEL_PATH)

Loading model...
Using device cuda.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Tokenizing....
Generating...
CPU times: user 43.4 s, sys: 44 s, total: 1min 27s
Wall time: 26.2 s


In [13]:
from llm_utils import extract_sql_code

print(extract_sql_code(response))


SELECT payments.paymentmethod, COUNT(*) AS COUNT FROM payments GROUP BY payments.paymentmethod ORDER BY COUNT DESC LIMIT 1;



In [5]:
from llm_utils import extract_sql_code
print(extract_sql_code(response))
query = extract_sql_code(response)

# Execute the query and format the output
try:
    result = postgres_connector.run_sql_as_df(query)
    print("Query executed successfully. The result is:")
    print(result.to_json())
except Exception as e:
    print(f"Error executing the query: {e}")


SELECT stores.store_name, products.product_name_en, COUNT(transactions.transaction_id) AS number_of_transactions
FROM stores
JOIN inventory ON stores.store_id = inventory.store_id
JOIN products ON inventory.product_id = products.product_id
JOIN transactions ON inventory.inventory_id = transactions.inventory_id
GROUP BY stores.store_name, products.product_name_en
ORDER BY stores.store_name, number_of_transactions DESC
LIMIT 1;

Query executed successfully. The result is:
{"store_name":{"0":"Store 1"},"product_name_en":{"0":"Tea"},"number_of_transactions":{"0":470}}


In [6]:
# Reading from the file
with open("sample_table_str.txt", 'r') as file:
    table_str = file.read()
system_prompt = f'You are an assistant to a SQL developer, you will help him interpret results of query execution. He gives a SQL database definition, SQL query he ran and the result it returns.\n- You go through the database definition for context.\n- You go through the SQL query and the question to prepare the response.\n- You use the sql execution result to provide a short sentence interpreting the results.'
user_message = f"Below is an SQL table definition: \n```sql\n{table_str}\n```\nBelow the executed query: \n```sql\n{query}\n``` \n\nBelow the results obtained after executing the query: \n`{result.to_json()}`\n\nAnswer the following question: \n{question}"
formatted_prompt = f"""
<s>[INST] <<SYS>>
{ system_prompt }
<</SYS>>

{ user_message } [/INST]
"""

print(formatted_prompt)


<s>[INST] <<SYS>>
You are an assistant to a SQL developer, you will help him interpret results of query execution. He gives a SQL database definition, SQL query he ran and the result it returns.
- You go through the database definition for context.
- You go through the SQL query and the question to prepare the response.
- You use the sql execution result to provide a short sentence interpreting the results.
<</SYS>>

Below is an SQL table definition: 
```sql

CREATE TABLE companies (
company_id SERIAL PRIMARY KEY,
company_name VARCHAR(255) -- company_id is the company unique ID
)

CREATE TABLE stores (
company_id integer, -- foreign key referencing companies.company_id
store_id VARCHAR(255) PRIMARY KEY, -- store_id is the store unique ID
store_name VARCHAR(255), -- name of the store
address text, -- address of the store
phone_number VARCHAR(20) -- phone number of the store
)

CREATE TABLE categories (
category_id VARCHAR(255) PRIMARY KEY, -- category_id is the category unique ID
categ

In [7]:
from llm_utils import prompt_llm_offline

response = prompt_llm_offline(prompt=formatted_prompt, max_length=2000)
print(response.split("[/INST]")[-1])

Loading model...
Using device mps.


Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.41s/it]


Tokenizing....
Generating...

 [INST] <<SYS>>
You are an assistant to a SQL developer, you will help him interpret results of query execution. He gives a SQL database definition, SQL query he ran and the result it returns.
- You go through the database definition for context.
- You go through the SQL query and the question to prepare the response.
- You use the sql execution result to provide a short sentence interpreting the results.
<</SYS>>

Below is an SQL table definition: 
```sql

CREATE TABLE companies (
company_id SERIAL PRIMARY KEY,
company_name VARCHAR(255) -- company_id is the company unique ID
)

CREATE TABLE stores (
company_id integer, -- foreign key referencing companies.company_id
store_id VARCHAR(255) PRIMARY KEY, -- store_id is the store unique ID
store_name VARCHAR(255), -- name of the store
address text, -- address of the store
phone_number VARCHAR(20) -- phone number of the store
)

CREATE TABLE categories (
category_id VARCHAR(255) PRIMARY KEY, -- category_id is t

## Manifest-ml deploying and using server side

In [8]:
from manifest import Manifest

manifest_client = Manifest(client_name="huggingface", client_connection="http://127.0.0.1:5001")

def get_sql(instruction: str, max_tokens: int = 300) -> str:
    prompt = formatter.format_prompt(instruction)
    res = manifest_client.run(prompt, max_tokens=max_tokens)
    return formatter.format_model_output(res)

In [9]:
print(manifest_client.client_pool.get_current_client().get_model_params())
print(manifest_client.client_pool.get_current_client().get_model_inputs())

{'model_name': 'NumbersStation/nsql-llama-2-7B', 'model_path': 'NumbersStation/nsql-llama-2-7B', 'client_name': 'huggingface'}
['temperature', 'max_tokens', 'n', 'top_p', 'top_k', 'repetition_penalty', 'do_sample']


In [10]:
sql = get_sql("How many products are sold in Store 001 ?")
print(sql)

prompt='CREATE TABLE companies (\n    company_id integer  primary key,\n    company_name character varying \n)\n\nCREATE TABLE stores (\n    company_id integer ,\n    store_id character varying  primary key,\n    store_name character varying ,\n    address text ,\n    phone_number character varying ,\n    foreign key (company_id) references companies(company_id)\n)\n\nCREATE TABLE categories (\n    category_id character varying  primary key,\n    category_name_en character varying \n)\n\nCREATE TABLE products (\n    product_id character varying  primary key,\n    article_id character varying ,\n    product_name_en character varying ,\n    uom character varying ,\n    buom character varying ,\n    category_id character varying ,\n    brand character varying ,\n    gtin character varying ,\n    varient_en character varying ,\n    foreign key (category_id) references categories(category_id)\n)\n\nCREATE TABLE inventory (\n    inventory_id integer  primary key,\n    current_stock integer ,

ReadTimeout: HTTPConnectionPool(host='127.0.0.1', port=5001): Read timed out. (read timeout=60)

In [12]:
print(postgres_connector.run_sql_as_df(sql))

ProgrammingError: (psycopg2.errors.UndefinedColumn) column t1.store_id does not exist
LINE 1: ...duct_id) FROM products AS t1 JOIN stores AS t2 ON t1.store_i...
                                                             ^
HINT:  Perhaps you meant to reference the column "t2.store_id".

[SQL: SELECT COUNT(DISTINCT t1.product_id) FROM products AS t1 JOIN stores AS t2 ON t1.store_id = t2.store_id WHERE t2.company_name = "A";]
(Background on this error at: https://sqlalche.me/e/14/f405)